# Foursquare API calls for Battle of the Neighborhood project

## 1. Load dataframe that contains coordinates info

In [1]:
import pandas as pd
import requests

In [2]:
data = pd.read_csv('wards_data.csv')

In [3]:
data.head()

,Code,Borough,Ward_Name,Year,Population,km2,Population_density,Lon,Lat,Wellbeing
0,E05000026,Barking and Dagenham,Abbey,2021,16938,1.279,13243.158720,0.077935,51.539711,-3.828524
1,E05000027,Barking and Dagenham,Alibon,2021,11323,1.361,8319.617928,0.148269,51.545589,-10.171974
2,E05000028,Barking and Dagenham,Becontree,2021,14891,1.284,11597.352020,0.118956,51.554534,-9.884958
3,E05000029,Barking and Dagenham,Chadwell Heath,2021,11297,3.380,3342.307692,0.139985,51.584751,-10.602072
4,E05000030,Barking and Dagenham,Eastbrook,2021,11032,3.454,3193.977997,0.173580,51.553647,-4.589297


## 2. API Credentials and parameters

In [5]:
CLIENT_ID = 'XXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXX' # your Foursquare Secret
ACCESS_TOKEN = 'XXXXX' # your FourSquare Access Token
VERSION = '20210427'
LIMIT = 100
radius = 750
OFFSET = 100
categoryIDs='4d4b7105d754a06374d81259' # food category

In [6]:
### Get calls for restaurants for each ward. Page 1 - first 100 restaurants in the 750 m radius

## 3. First call for restaurants

In [ ]:
missing_codes = []
venues_list=[]

for name, lat, lng in zip(data.Code, data.Lat, data.Lon):
    print(name)
            
#create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryIDs)
    for attempt in range(10): # 10 attempts
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']  # call and get fields       
#return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print('retrying')
                                
        else:
            break
    else:   # successfully ended loop, no break encountered
        missing_codes.append(name)
            

nearby_restaurants = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_restaurants.columns = ['Code', 
                  'Ward Lat', 
                  'Ward Lon', 
                  'Venue', 
                  'Venue Lat', 
                  'Venue Lon', 
                  'Venue Category']
       

In [8]:
nearby_restaurants.to_csv('restaurants.csv', index=False)

In [9]:
restaurants = pd.read_csv('restaurants.csv')


In [10]:
restaurants.head()

,Code,Ward Lat,Ward Lon,Venue,Venue Lat,Venue Lon,Venue Category
0,E05000026,51.539711,0.077935,Cristina's,51.536523,0.076672,Steakhouse
1,E05000026,51.539711,0.077935,Nando's,51.539780,0.082297,Portuguese Restaurant
2,E05000026,51.539711,0.077935,Subway,51.538000,0.081319,Sandwich Place
3,E05000026,51.539711,0.077935,Domino's Pizza,51.539833,0.082223,Pizza Place
4,E05000026,51.539711,0.077935,KFC,51.539117,0.080792,Fast Food Restaurant


## 4. Second call for restaurants

Some wards have more than 100 restaurants, and since we limit our calls to 100 results, we can't get all information. Let's see which wards have more than 100 restaurant and make calls only for them with higher limits

In [11]:
rest_per_ward = restaurants.Code.value_counts()
rest_100 = rest_per_ward[rest_per_ward == 100].index.tolist()
rest_100 = pd.DataFrame(rest_100)
rest_100.columns = ['Code']

rest_100.head()

,Code
0,E05000370
1,E05000587
2,E05000536
3,E05000588
4,E05000263


In [12]:
wards_100 = data[data.Code.isin(rest_100.Code)]
wards_100.head()

,Code,Borough,Ward_Name,Year,Population,km2,Population_density,Lon,Lat,Wellbeing
103,E05000129,Camden,Bloomsbury,2021,12380,1.021,12125.36729,-0.131436,51.521987,3.403726
112,E05000138,Camden,Holborn and Covent Garden,2021,15374,1.201,12800.99917,-0.117735,51.519008,-0.252403
115,E05000141,Camden,King's Cross,2021,13321,0.610,21837.70492,-0.121304,51.527145,0.387644
213,E05000239,Hackney,Haggerston,2021,19199,1.243,15445.69590,-0.075513,51.530331,-5.920721
214,E05000240,Hackney,Hoxton,2021,18587,0.839,22153.75447,-0.087929,51.531135,-5.711618


In [13]:
missing_codes = []
venues_list=[]
LIMIT = 100
OFFSET = 100
for name, lat, lng in zip(wards_100.Code, wards_100.Lat, wards_100.Lon):
    print(name)
            
#create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            OFFSET,
            categoryIDs)
    for attempt in range(10): # 10 attempts
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']  # call and get fields       
#return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            print('retrying')
                                
        else:
            break
    else:   # successfully ended loop, no break encountered
        missing_codes.append(name)
            

restaurants_100 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
restaurants_100.columns = ['Code', 
                  'Ward Lat', 
                  'Ward Lon', 
                  'Venue', 
                  'Venue Lat', 
                  'Venue Lon', 
                  'Venue Category']
       

E05000129
E05000138
E05000141
E05000239
E05000240
E05000263
E05000366
E05000370
E05000383
E05000384
E05000386
E05000390
E05000394
E05000416
E05000536
E05000587
E05000588
E05000631
E05000632
E05000636
E05000638
E05000641
E05000646
E05000647
E05000649


In [14]:
restaurants_100.Code.value_counts()

E05000649    100
E05000536     66
E05000386     57
E05000641     56
E05000383     49
E05000638     43
E05000632     38
E05000647     37
E05000129     37
E05000588     35
E05000390     31
E05000636     30
E05000141     29
E05000366     24
E05000239     21
E05000394     19
E05000416     12
E05000631      7
E05000587      6
E05000263      5
E05000138      5
E05000646      4
E05000240      3
E05000370      1
Name: Code, dtype: int64

Only one ward has more that 200 restaurants - upon checking the number turned out to be 201, so we will not make any extra call for it

## 5. Append calls data

In [15]:
all_restaurants = restaurants.append(restaurants_100)

In [16]:
all_restaurants.head()

,Code,Ward Lat,Ward Lon,Venue,Venue Lat,Venue Lon,Venue Category
0,E05000026,51.539711,0.077935,Cristina's,51.536523,0.076672,Steakhouse
1,E05000026,51.539711,0.077935,Nando's,51.539780,0.082297,Portuguese Restaurant
2,E05000026,51.539711,0.077935,Subway,51.538000,0.081319,Sandwich Place
3,E05000026,51.539711,0.077935,Domino's Pizza,51.539833,0.082223,Pizza Place
4,E05000026,51.539711,0.077935,KFC,51.539117,0.080792,Fast Food Restaurant


In [17]:
all_restaurants.to_csv('all_restaurants.csv', index=False)